In [ ]:
from pynq.overlays.base import BaseOverlay # Video: https://youtu.be/iofNuJ9xQvs
import time
import ipywidgets as widgets
base = BaseOverlay("base.bit")

In [2]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Set GPIO pins to all 0
void clear_gpio(){
    for(int i=0; i<8; i++){
        gpio pin_out = gpio_open(i);
        gpio_set_direction(pin_out, GPIO_OUT);
        gpio_write(pin_out, 0);
    }
}


//Function to turn on/off a selected pin of PMODB
int write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
        return 0;
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
    return 1;
}

/* 
//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}
*/

In [31]:
def pwm_for_duration(duty, rate, duration, pin):
    start=time.time()
    if duty>=100:
        ret = write_gpio(pin, 1)
        time.sleep(duration)
        ret = write_gpio(pin, 0)
        return
    elif duty==0:
        time.sleep(duration)
        return
    while start+duration>time.time():
        ret = write_gpio(pin, 1)
        time.sleep((1/rate)*(duty/100.0))
        ret = write_gpio(pin, 0)
        time.sleep((1/rate)-(1/rate)*(duty/100.0))

clear_gpio()
for i in range(11):
    pwm_for_duration(10*i, 50, 1, 2)
    time.sleep(0.2) # so you can visually see when it changes



In [35]:
clear_gpio()
for i in range(4):
    pwm_for_duration(25*(i+1), 50, 1, 2)
    #time.sleep(0.05) # so you can visually see when it changes


In [4]:
import asyncio
btns = base.btns_gpio

In [36]:
pin = 3
done = False

# using await asyncio.sleep
async def pwm_for_duration(duty, rate, duration, pin):
    global done
    start=time.time()
    if duty==100:
        ret = write_gpio(pin, 1)
        await asyncio.sleep(duration)
        ret = write_gpio(pin, 0)
        return
    elif duty==0:
        await asyncio.sleep(duration)
        return
    while start+duration>time.time() and not done:
        ret = write_gpio(pin, 1)
        await asyncio.sleep((1/rate)*(duty/100.0))
        ret = write_gpio(pin, 0)
        await asyncio.sleep((1/rate)-(1/rate)*(duty/100.0))

async def get_btns(_loop):
    global pin, done
    while True:
        await asyncio.sleep(0.01)
        res = btns.read()
        if res & 0b0001:
            pin = 3
        if res & 0b0010:
            pin = 2
        if res & 0b0100:
            pin = 1
        if res & 0b1000:
            done=True
            await asyncio.sleep(0.1)
            _loop.stop()
            
async def run_led():
    global pin
    while True:
        await pwm_for_duration(25, 50, 1, pin)
        await asyncio.sleep(1)


loop = asyncio.new_event_loop()
loop.create_task(run_led())
loop.create_task(get_btns(loop))
loop.run_forever()
loop.close()
